# PROBLEMA DE NEGÓCIO: SEGMENTAÇÃO DE CLIENTES

# INSTALAÇÃO E CARREGAMENTO DOS PACOTES

In [ ]:
# Importando os pacotes 
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib as mat
import matplotlib.pyplot as plt
import colorsys
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import homogeneity_completeness_v_measure
from sklearn.preprocessing import MinMaxScaler
plt.style.use('seaborn-talk')
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# Carregando os dados
df = pd.read_csv('dataset.csv', sep = ',', encoding = 'utf-8')

# ANÁLISE EXPLORATÓRIA, LIMPEZA E TRANFORMAÇÃO

In [ ]:
# Visualizando os dados
df.head(5)

In [ ]:
# Verificando o tipo dos dados
df.dtypes

In [ ]:
# Visualizando as dimensões
df.shape

In [ ]:
# Ajustando a coluna ID
df.reset_index(inplace=True, drop=False)
df.drop(columns=["ID"],inplace=True)

In [ ]:
# Renomeando as colunas
colunas = ['Id', 'Sexo', 'Estado_Civil', 'Idade', 'Educacao', 'Renda', 'Ocupacao', 'Tamanho_Cidade']
df.columns = colunas
df.head()

In [ ]:
# Convertendo as variáveis Id, Sexo, Estado_Civil, Educacao, Ocupacacao e Tamanho_Cidade para categorias
df['Id'] = pd.Categorical(df['Id'])
df['Sexo'] = pd.Categorical(df['Sexo'])
df['Estado_Civil'] = pd.Categorical(df['Estado_Civil'])
df['Educacao'] = pd.Categorical(df['Educacao'])
df['Ocupacao'] = pd.Categorical(df['Ocupacao'])
df['Tamanho_Cidade'] = pd.Categorical(df['Tamanho_Cidade'])

# Verificando o tipo dos dados
df.dtypes

In [ ]:
# Criando uma cópia do dataframne
df1 = df.copy()

In [ ]:
# Resumo
print("Linhas: ", df1.shape[0])
print("Colunas: ", df1.shape[1])
print("\nVariáveis: \n", df1.columns.tolist())
print("\nValores Ausentes: \n" , df1.isnull().sum())
print("\nValores Únicos: \n", df1.nunique())

In [ ]:
# Colunas numéricas (quantitativas)
colunas_numericas = ['Idade', 'Renda']

In [ ]:
# Colunas categóricas
colunas_categoricas = ['Id', 'Sexo', 'Estado_Civil', 'Educacao', 'Ocupacao', 'Tamanho_Cidade']

In [ ]:
# Verifica se o total de variáveis está coberto nos objetos anteriores
len(colunas_numericas) + len(colunas_categoricas) == 8

In [ ]:
# Dataframes com os tipos diferentes de variáveis
df_num = df1[colunas_numericas]
df_cat = df1[colunas_categoricas]

In [ ]:
# Sumário estatístico das variáveis numéricas
sumario_num = pd.DataFrame(index = df_num.columns)
sumario_num['Tipo de Dado'] = df_num.dtypes.values
sumario_num['Registros Não Nulos'] = df_num.count().values
sumario_num['Registros Não Zero'] = df_num.astype(bool).sum(axis = 0)
sumario_num['% Populado'] = round(sumario_num['Registros Não Nulos'] / df_num.shape[0]*100,2)
sumario_num['Valores Únicos'] = df_num.nunique().values
sumario_num['Mean'] = round(df_num.mean(),2)
sumario_num['Std'] = round(df_num.std(),2)
sumario_num['Min'] = round(df_num.min(),2)
sumario_num['Max'] = round(df_num.max(),2)
sumario_num

In [ ]:
# Sumário estatístico das variáveis categóricas
sumario_cat = pd.DataFrame(index = df_cat.columns)
sumario_cat['Tipo de Dado'] = df_cat.dtypes.values
sumario_cat['Registros Não Nulos'] = df_cat.count().values
sumario_cat['% Populado'] = round(sumario_cat['Registros Não Nulos'] / df_cat.shape[0]*100,2)
sumario_cat['# Valores Únicos'] = df_cat.nunique().values

# Adiciona mais uma coluna com valores mais comuns
temp = []
for coluna in colunas_categoricas:
    temp.append(df_cat[coluna].value_counts().idxmax())
sumario_cat['Valores Mais Comuns'] = temp
sumario_cat

# VISUALIZAÇÃO DAS VARIÁVEIS

In [ ]:
# Variável 2 - Sexo (0= Masculino  1= Feminino)
sexo = df1['Sexo'].value_counts().rename_axis('Sexo').reset_index(name = 'Total')
sexo

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig1 = sns.countplot(x = 'Sexo', data = df1, orient = 'v')
plt.title("Sexo")

In [ ]:
# Variável 3 - Estado_Civil (0= Solteiro  1= Não Solteiro [Casado/Divorciado/Separado/Viúvo])
estado_civil = df1['Estado_Civil'].value_counts().rename_axis('Estado_Civil').reset_index(name = 'Total')
estado_civil

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig2 = sns.countplot(x = 'Estado_Civil', data = df1, orient = 'v')
plt.title("Estado Civil")

In [ ]:
# Variável 4 - Idade
df1['Idade'].describe().round(2)

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (20, 10))
fig3 = sns.boxplot(x = 'Idade',  data = df1, color='lightblue', orient = 'v')
plt.title("Boxplot Idade")

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (20, 10))
fig4 = sns.histplot(df1, x = 'Idade', color= 'blue', kde = True, bins = 8)
plt.title("Idade")

In [ ]:
# Calculando a Assimetria e Curtose
assimetria_idade = scipy.stats.skew(df1['Idade'])
curtose_idade = scipy.stats.kurtosis(df1['Idade'])
print(assimetria_idade)
print(curtose_idade)

In [ ]:
# Variável 6 - Educacao (0= Outro/Desconhecido  1= Ensino Médio  2= Universitário  3= Graduado)
educacao = df1['Educacao'].value_counts().rename_axis('Educacao').reset_index(name = 'Total')
educacao

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig5 = sns.countplot(x = 'Educacao', data = df1, orient = 'v')
plt.title("Educação")

In [ ]:
def percentile(n):

    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

resumo_educacao_renda = df1.groupby(['Educacao'], sort=True).agg(count= ('Renda', 'count'), mean=('Renda','mean'),                                                                 
desvpad=('Renda','std'),
minimo=('Renda','min'),
p25=('Renda',percentile(25)),
p50=('Renda',percentile(50)),
p75=('Renda',percentile(75)),
maximo=('Renda','max')).reset_index().round(2)

resumo_educacao_renda

In [ ]:
# Criando a coluna com o nome das categorias
for item in range(0,len(resumo_educacao_renda['Educacao']),1):
   
    if resumo_educacao_renda['Educacao'][item] == 0:
        resumo_educacao_renda['Educacao_cat']= 'Outro'
    elif resumo_educacao_renda['Educacao'][item] == 1:
        resumo_educacao_renda['Educacao_cat']= 'Ensino Médio'
    elif resumo_educacao_renda['Educacao'][item] == 2:
        resumo_educacao_renda['Educacao_cat']= 'Universitário'
    elif resumo_educacao_renda['Educacao'][item] == 3:
        resumo_educacao_renda['Educacao_cat']= 'Graduado'

In [ ]:
for item in range(0,len(resumo_educacao_renda['Educacao']),1):
   
    if resumo_educacao_renda['Educacao'][item] == 0:
        resumo_educacao_renda['Educacao_cat'][item]= 'Outro'
    elif resumo_educacao_renda['Educacao'][item] == 1:
        resumo_educacao_renda['Educacao_cat'][item]= 'Ensino Médio'
    elif resumo_educacao_renda['Educacao'][item] == 2:
        resumo_educacao_renda['Educacao_cat'][item]= 'Universitário'
    elif resumo_educacao_renda['Educacao'][item] == 3:
        resumo_educacao_renda['Educacao_cat'][item]= 'Graduado'

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig6 = sns.barplot(x = resumo_educacao_renda['Educacao_cat'], y = resumo_educacao_renda['mean'], orient = 'v',  palette="Blues_d", )
plt.title("Média de Renda por Educação")

In [ ]:
# Variável 7 - Renda
df1['Renda'].describe().round(2)

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (20, 10))
fig7 = sns.boxplot(x = 'Renda',  data = df1, color='lightblue', orient = 'v')
plt.title("Boxplot Renda")

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (20, 10))
fig8 = sns.histplot(df1, x = 'Renda', color= 'blue', kde = True, bins = 12)
plt.title("Renda")

In [ ]:
# Calculando a Assimetria e Curtose
assimetria_renda = scipy.stats.skew(df1['Renda'])
curtose_renda = scipy.stats.kurtosis(df1['Renda'])
print(assimetria_renda)
print(curtose_renda)

In [ ]:
# Variável 8 - Ocupacao (0= Desempregado  1= Funcionário  2= Gestão/Autônomo)
ocupacao = df1['Ocupacao'].value_counts().rename_axis('Ocupacao').reset_index(name = 'Total')
ocupacao

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig9 = sns.countplot(x = 'Ocupacao', data = df1, orient = 'v')
plt.title("Ocupação")

In [ ]:
def percentile(n):

    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

resumo_ocupacao_renda = df1.groupby(['Ocupacao'], sort=True).agg(count= ('Renda', 'count'), mean=('Renda','mean'),                                                                 
desvpad=('Renda','std'),
minimo=('Renda','min'),
p25=('Renda',percentile(25)),
p50=('Renda',percentile(50)),
p75=('Renda',percentile(75)),
maximo=('Renda','max'),).reset_index().round(2)

resumo_ocupacao_renda

In [ ]:
# Criando a coluna com o nome das categorias
for item in range(0,len(resumo_ocupacao_renda['Ocupacao']),1):
   
    if resumo_ocupacao_renda['Ocupacao'][item] == 0:
        resumo_ocupacao_renda['Ocupacao_cat']= 'Desempregado'
    elif resumo_ocupacao_renda['Ocupacao'][item] == 1:
        resumo_ocupacao_renda['Ocupacao_cat']= 'Funcionário'
    elif resumo_ocupacao_renda['Ocupacao'][item] == 2:
        resumo_ocupacao_renda['Ocupacao_cat']= 'Gestão/Autônomo'

In [ ]:
for item in range(0,len(resumo_ocupacao_renda['Ocupacao']),1):
   
    if resumo_ocupacao_renda['Ocupacao'][item] == 0:
        resumo_ocupacao_renda['Ocupacao_cat'][item]= 'Desempregado'
    elif resumo_ocupacao_renda['Ocupacao'][item] == 1:
        resumo_ocupacao_renda['Ocupacao_cat'][item]= 'Funcionário'
    elif resumo_ocupacao_renda['Ocupacao'][item] == 2:
        resumo_ocupacao_renda['Ocupacao_cat'][item]= 'Gestão/Autônomo'

In [ ]:
resumo_ocupacao_renda.head()

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig10 = sns.barplot(x = resumo_ocupacao_renda['Ocupacao_cat'], y = resumo_ocupacao_renda['mean'], orient = 'v',palette="Blues_d") 
plt.title("Média de Renda por Ocupação")

In [ ]:
# Variável 9 - Tamanho_Cidade (0= Pequena  1= Média  2= Grande)
tamanho_cidade = df1['Tamanho_Cidade'].value_counts().rename_axis('Tamanho_cidade').reset_index(name = 'Total')
tamanho_cidade

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig11 = sns.countplot(x = 'Tamanho_Cidade', data = df1, orient = 'v')
plt.title("Tamanho da Cidade")

In [ ]:
def percentile(n):

    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

resumo_cidade_renda = df1.groupby(['Tamanho_Cidade'], sort=True).agg(count= ('Renda', 'count'), mean=('Renda','mean'),                                                                 
desvpad=('Renda','std'),
minimo=('Renda','min'),
p25=('Renda',percentile(25)),
p50=('Renda',percentile(50)),
p75=('Renda',percentile(75)),
maximo=('Renda','max'),).reset_index().round(2)

resumo_cidade_renda

In [ ]:
# Criando a coluna com o nome das categorias
for item in range(0,len(resumo_cidade_renda['Tamanho_Cidade']),1):
   
    if resumo_cidade_renda['Tamanho_Cidade'][item] == 0:
        resumo_cidade_renda['Tamanho_Cidade_cat']= 'Pequena'
    elif resumo_cidade_renda['Tamanho_Cidade'][item] == 1:
        resumo_cidade_renda['Tamanho_Cidade_cat']= 'Média'
    elif resumo_cidade_renda['Tamanho_Cidade'][item] == 2:
        resumo_cidade_renda['Tamanho_Cidade_cat']= 'Grande'

In [ ]:
for item in range(0,len(resumo_cidade_renda['Tamanho_Cidade']),1):
   
    if resumo_cidade_renda['Tamanho_Cidade'][item] == 0:
        resumo_cidade_renda['Tamanho_Cidade_cat'][item]= 'Pequena'
    elif resumo_cidade_renda['Tamanho_Cidade'][item] == 1:
        resumo_cidade_renda['Tamanho_Cidade_cat'][item]= 'Média'
    elif resumo_cidade_renda['Tamanho_Cidade'][item] == 2:
        resumo_cidade_renda['Tamanho_Cidade_cat'][item]= 'Grande'

In [ ]:
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)
plt.figure(figsize = (10, 6))
fig12 = sns.barplot(x = resumo_cidade_renda['Tamanho_Cidade_cat'], y = resumo_cidade_renda['mean'], orient = 'v', palette="Blues_d") 
plt.title("Média de Renda por Tamanho da Cidade")

# MODELAGEM PREDITIVA

# K-MEANS

# Segmentação 1 - Encontrando o Valor Ideal de Clusters

In [ ]:
# Lista do WCSS
wcss_df1 = []

In [ ]:
# Loop para testar os valores de K
for n in range(2, 11):
    modelo = (KMeans(n_clusters = n,
                     init = 'k-means++', 
                     n_init = 10,
                     max_iter = 300, 
                     tol = 0.0001,  
                     random_state = 111, 
                     algorithm = 'elkan'))
    modelo.fit(df1)
    wcss_df1.append(modelo.inertia_)

In [ ]:
wcss_df1

In [ ]:
modelo.inertia_

In [ ]:
# Plot
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(2 , 11) , wcss_df1 , 'o', color = "red")
plt.plot(np.arange(2 , 11) , wcss_df1 , '-' , alpha = 0.5, color = 'blue')
plt.xlabel('Número de Clusters') , plt.ylabel('WCSS')
plt.title('WCSS x Número de Clusters')
plt.show()

# Segmentação 1 - Construindo e Treinando o Modelo

In [ ]:
# Criação do modelo
modelo_v1 = KMeans(n_clusters = 4, 
                     init = 'k-means++', 
                     n_init = 10, 
                     max_iter = 300, 
                     tol = 0.0001,   
                     algorithm = 'elkan')

In [ ]:
# Treinamento do modelo
modelo_v1.fit(df1)

In [ ]:
# Extração dos labels
labels1 = modelo_v1.labels_
labels1

In [ ]:
# Extração dos centróides
centroids1 = modelo_v1.cluster_centers_
centroids1

In [ ]:
# Lista para receber as métricas
HCVs = []

# Um resultado de cluster satisfaz a homogeneidade se todos os seus clusters contiverem apenas pontos de 
# dados que são membros de uma única classe.

# Um resultado de cluster satisfaz a completude se todos os pontos de dados que são membros 
# de uma determinada classe são elementos do mesmo cluster.

# Ambas as pontuações têm valores positivos entre 0,0 e 1,0, sendo desejáveis valores maiores.

# Medida V é a média entre homogeneidade e completude.

previsoes = modelo_v1.fit_predict(df1)
    
# Calculando as métricas
HCVs.append(homogeneity_completeness_v_measure(labels1, previsoes))

HCVs

In [ ]:
# Inertia = Soma das distâncias das amostras para o seu centro de agrupamento mais próximo.

# Lista para as métricas
valores_metrica = []

valores_metrica.append(modelo_v1.inertia_)
valores_metrica

In [ ]:
from sklearn.metrics import silhouette_score
# ?silhouette_score

In [ ]:
# Silhouette Score
silhouette_score(df1, labels1, metric = 'euclidean')

In [ ]:
# Converte o array para dataframe
df_labels = pd.DataFrame(labels1)
df_labels

In [ ]:
# Vamos fazer o merge de df2 e os labels (clusters) encontrados pelo modelo
df_final = df1.merge(df_labels, left_index = True, right_index = True)
df_final.rename(columns = {0:"Cluster"}, inplace = True)

In [ ]:
df_final

In [ ]:
cluster = df_final['Cluster'].value_counts().rename_axis('Cluster').reset_index(name = 'Total')
cluster

# Segmentação 2 - Construindo e Treinando o Modelo

In [ ]:
from sklearn.decomposition import PCA
# Aplicando redução de dimensionalidade
pca = PCA(n_components = 2).fit_transform(df1)

In [ ]:
# Determinando um range de K
k_range = range(1,10)

In [ ]:
# Aplicando o modelo K-Means para cada valor de K (esta célula pode levar bastante tempo para ser executada)
k_means_var = [KMeans(n_clusters = k).fit(pca) for k in k_range]

In [ ]:
# Ajustando o centróide do cluster para cada modelo
centroids = [X.cluster_centers_ for X in k_means_var]

In [ ]:
# Calculando a distância euclidiana de cada ponto de dado para o centróide
from scipy.spatial.distance import cdist, pdist
k_euclid = [cdist(pca, cent, 'euclidean') for cent in centroids]
dist = [np.min(ke, axis = 1) for ke in k_euclid]

In [ ]:
# Soma dos quadrados das distâncias dentro do cluster
soma_quadrados_intra_cluster = [sum(d**2) for d in dist]

In [ ]:
# Soma total dos quadrados
soma_total = sum(pdist(pca)**2)/pca.shape[0]

In [ ]:
# Soma dos quadrados entre clusters
soma_quadrados_inter_cluster = soma_total - soma_quadrados_intra_cluster

In [ ]:
k_range1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
celbow = pd.DataFrame(soma_quadrados_inter_cluster)
celbow['soma_total'] = soma_total
celbow['range'] = k_range1
colunas2 = ['inter_cluster', 'soma_total', 'range']
celbow.columns = colunas2
celbow['variancia'] = (celbow['inter_cluster'] / celbow['soma_total']) * 100
celbow

In [ ]:
# Curva de Elbow
plt.figure(1 , figsize = (15,6))
plt.plot(k_range, soma_quadrados_inter_cluster/soma_total * 100, 'o', color= 'red')
plt.plot(k_range, soma_quadrados_inter_cluster/soma_total * 100, '-', color= 'blue')
plt.grid(True)
plt.xlabel('Número de Clusters')
plt.ylabel('Percentual de Variância Explicada')
plt.title('Variância Explicada x Número de Cluster')

In [ ]:
# Criação do modelo
modelo_v2 = KMeans(n_clusters = 4, 
                     init = 'k-means++', 
                     n_init = 10, 
                     max_iter = 300, 
                     tol = 0.0001,   
                     algorithm = 'elkan')

In [ ]:
# Treinamento do modelo
modelo_v2.fit(pca)

In [ ]:
# Extração dos labels
labels2 = modelo_v2.labels_
labels2

In [ ]:
# Lista para receber as métricas
HCVs2 = []

# Um resultado de cluster satisfaz a homogeneidade se todos os seus clusters contiverem apenas pontos de 
# dados que são membros de uma única classe.

# Um resultado de cluster satisfaz a completude se todos os pontos de dados que são membros 
# de uma determinada classe são elementos do mesmo cluster.

# Ambas as pontuações têm valores positivos entre 0,0 e 1,0, sendo desejáveis valores maiores.

# Medida V é a média entre homogeneidade e completude.

previsoes2 = modelo_v2.fit_predict(pca)
    
# Calculando as métricas
HCVs2.append(homogeneity_completeness_v_measure(labels2, previsoes2))

HCVs2

In [ ]:
# Inertia = Soma das distâncias das amostras para o seu centro de agrupamento mais próximo.

# Lista para as métricas
valores_metrica2 = []

valores_metrica2.append(modelo_v2.inertia_)
valores_metrica2

In [ ]:
# Silhouette Score
silhouette_score(pca, labels2, metric = 'euclidean')

In [ ]:
# Converte o array para dataframe
df_labels2 = pd.DataFrame(labels2)
df_labels2

In [ ]:
# Vamos fazer o merge de pca e os labels (clusters) encontrados pelo modelo
df_final2 = df1.merge(df_labels2, left_index = True, right_index = True)
df_final2.rename(columns = {0:"Cluster"}, inplace = True)

In [ ]:
df_final2

In [ ]:
cluster2 = df_final2['Cluster'].value_counts().rename_axis('Cluster').reset_index(name = 'Total')
cluster2

In [ ]:
resumo_cluster = df_final2.groupby(['Cluster'], sort=True).agg(Media_Idade=('Idade','mean'), 
                                                               Media_Renda= ('Renda', 'mean')).reset_index().round(2)                                                             

resumo_cluster

In [ ]:
# Calcula a quantidade de clientes de cada genero por cluster
sexo_cluster = df_final2.groupby('Cluster')['Sexo'].value_counts()
sexo_cluster = pd.DataFrame(sexo_cluster)
sexo_cluster

In [ ]:
sexo_cluster.shape

In [ ]:
data2 = pd.melt(sexo_cluster.reset_index(),id_vars=['Cluster', 'Sexo'], value_name='Total')
#data2.drop(columns=["variable"],inplace=True)

In [ ]:
data2

In [ ]:
for item in range(0,len(data2['Cluster']),1):
   
    if data2['Cluster'][item] == 0 and data2['Sexo'][item] == 0:
        data2['Masculino']= data2['Total']
    elif data2['Cluster'][item] == 0 and data2['Sexo'][item] == 1:
        data2['Feminino']= data2['Total']

In [ ]:
data2['Masculino']= 'NAN'
data2['Feminino']= 'NAN'

In [ ]:
for index, row in data2.iterrows():
    if row['Cluster'] == 0 and row['Sexo']== 0:
        data2.loc[index,'Masculino'] =  data2.loc[index,'Total']
    elif row['Cluster'] == 1 and row['Sexo']== 0:
        data2.loc[index,'Masculino'] =  data2.loc[index,'Total']
    elif row['Cluster'] == 2 and row['Sexo']== 0:
        data2.loc[index,'Masculino'] =  data2.loc[index,'Total']   
    elif row['Cluster'] == 3 and row['Sexo']== 0:
        data2.loc[index,'Masculino'] =  data2.loc[index,'Total']
    elif row['Cluster'] == 0 and row['Sexo']== 1:
        data2.loc[index,'Feminino'] =  data2.loc[index,'Total']
    elif row['Cluster'] == 1 and row['Sexo']== 1:
        data2.loc[index,'Feminino'] =  data2.loc[index,'Total']
    elif row['Cluster'] == 2 and row['Sexo']== 1:
        data2.loc[index,'Feminino'] =  data2.loc[index,'Total']   
    elif row['Cluster'] == 3 and row['Sexo']== 1:
        data2.loc[index,'Feminino'] =  data2.loc[index,'Total']

In [ ]:
data2.drop(columns=["Sexo", "Total"],inplace=True)

In [ ]:
data2

In [ ]:
masculino = [472, 318,222,74]
feminino = [483, 139, 263, 29]

In [ ]:
resumo_cluster['Maculino'] = pd.Series(masculino).values
resumo_cluster['Feminino'] = pd.Series(feminino).values

In [ ]:
# Calcula a quantidade de clientes de cada estado civil por cluster
estado_civil_cluster = df_final2.groupby('Cluster')['Estado_Civil'].value_counts()
estado_civil_cluster = pd.DataFrame(estado_civil_cluster)
estado_civil_cluster

In [ ]:
solteiro = [450, 258, 236, 63]
nao_solteiro = [505, 199, 247, 42]

In [ ]:
resumo_cluster['Solteiro'] = pd.Series(solteiro).values
resumo_cluster['Nao_Solteiro'] = pd.Series(nao_solteiro).values

In [ ]:
# Calcula a quantidade de clientes de cada nível de formação por cluster
educacao_cluster = df_final2.groupby('Cluster')['Educacao'].value_counts()
educacao_cluster = pd.DataFrame(educacao_cluster)
educacao_cluster

In [ ]:
outro= [125, 53, 93, 16]
ensino_medio= [714, 279, 354, 39]
universitario= [104, 109, 34, 44]
graduado= [12, 16, 4, 4]

In [ ]:
resumo_cluster['Outro'] = pd.Series(outro).values
resumo_cluster['Ensino_Medio'] = pd.Series(ensino_medio).values
resumo_cluster['Universitario'] = pd.Series(universitario).values
resumo_cluster['Graduado'] = pd.Series(graduado).values

In [ ]:
# Calcula a quantidade de clientes de cada ocupação por cluster
ocupacao_cluster = df_final2.groupby('Cluster')['Ocupacao'].value_counts()
ocupacao_cluster = pd.DataFrame(ocupacao_cluster)
ocupacao_cluster

In [ ]:
desempregado= [190, 31, 412, 0]
funcionario= [739, 281, 70, 23]
gestao_autonomo= [26, 145, 3, 80]

In [ ]:
resumo_cluster['Desempregado'] = pd.Series(desempregado).values
resumo_cluster['Funcionario'] = pd.Series(funcionario).values
resumo_cluster['Gestao_Autonomo'] = pd.Series(gestao_autonomo).values

In [ ]:
# Calcula a quantidade de clientes de cada tamanho de cidade por cluster
cidade_cluster = df_final2.groupby('Cluster')['Tamanho_Cidade'].value_counts()
cidade_cluster = pd.DataFrame(cidade_cluster)
cidade_cluster

In [ ]:
pequena= [481, 69, 432, 7]
media= [266, 212, 26, 40]
grande= [208, 176, 27, 56]

In [ ]:
resumo_cluster['Pequena'] = pd.Series(pequena).values
resumo_cluster['Media'] = pd.Series(media).values
resumo_cluster['Grande'] = pd.Series(grande).values
resumo_cluster

# COMPARATIVO ENTRE OS CLUSTERS

In [ ]:
# Configurando o tema
sns.set_theme(style = 'darkgrid', font='sans-serif', font_scale=1)

# Configurando a figura matplotlib
f, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, 
    ax12, ax13, ax14, ax15, ax16) = plt.subplots(16, 1, figsize=(10, 30), sharex=True)

#Construindo os gráficos
fig13= sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Media_Idade'] , palette= "deep", ax=ax1)
ax1.axhline(0, color="k", clip_on=False)
ax1.set_ylabel("Média de Idade")
for p in ax1.patches:
    ax1.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')

sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Media_Renda'] , palette= "deep", ax=ax2)
ax2.axhline(0, color="k", clip_on=False)
ax2.set_ylabel("Média da Renda")
for p in ax2.patches:
    ax2.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')


sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Maculino'] , palette= "deep", ax=ax3)
ax3.axhline(0, color="k", clip_on=False)
ax3.set_ylabel("Maculino")
for p in ax3.patches:
    ax3.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')

sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Feminino'] , palette= "deep", ax=ax4)
ax4.axhline(0, color="k", clip_on=False)
ax4.set_ylabel("Feminino")
for p in ax4.patches:
    ax4.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Solteiro'] , palette= "deep", ax=ax5)
ax5.axhline(0, color="k", clip_on=False)
ax5.set_ylabel("Solteiro")
for p in ax5.patches:
    ax5.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Nao_Solteiro'] , palette= "deep", ax=ax6)
ax6.axhline(0, color="k", clip_on=False)
ax6.set_ylabel("Não Solteiro")
for p in ax6.patches:
    ax6.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Outro'] , palette= "deep", ax=ax7)
ax7.axhline(0, color="k", clip_on=False)
ax7.set_ylabel("Outro")
for p in ax7.patches:
    ax7.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Ensino_Medio'] , palette= "deep", ax=ax8)
ax8.axhline(0, color="k", clip_on=False)
ax8.set_ylabel("Ensino Médio")
for p in ax8.patches:
    ax8.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Universitario'] , palette= "deep", ax=ax9)
ax9.axhline(0, color="k", clip_on=False)
ax9.set_ylabel("Universitário")
for p in ax9.patches:
    ax9.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Graduado'] , palette= "deep", ax=ax10)
ax10.axhline(0, color="k", clip_on=False)
ax10.set_ylabel("Graduado")
for p in ax10.patches:
    ax10.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Desempregado'] , palette= "deep", ax=ax11)
ax11.axhline(0, color="k", clip_on=False)
ax11.set_ylabel("Desempregado")
for p in ax11.patches:
    ax11.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Funcionario'] , palette= "deep", ax=ax12)
ax12.axhline(0, color="k", clip_on=False)
ax12.set_ylabel("Funcionario")
for p in ax12.patches:
    ax12.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')

sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Gestao_Autonomo'] , palette= "deep", ax=ax13)
ax13.axhline(0, color="k", clip_on=False)
ax13.set_ylabel("Gestão ou Autônomo")
for p in ax13.patches:
    ax13.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')

sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Pequena'] , palette= "deep", ax=ax14)
ax14.axhline(0, color="k", clip_on=False)
ax14.set_ylabel("Cidade Pequena")
for p in ax14.patches:
    ax14.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Media'] , palette= "deep", ax=ax15)
ax15.axhline(0, color="k", clip_on=False)
ax15.set_ylabel("Cidade Média")
for p in ax15.patches:
    ax15.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
sns.barplot(x=resumo_cluster['Cluster'], y= resumo_cluster['Grande'] , palette= "deep", ax=ax16)
ax16.axhline(0, color="k", clip_on=False)
ax16.set_ylabel("Cidade Grande")
for p in ax16.patches:
    ax16.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
    
# Finalizando os gráficos
sns.despine(bottom=True)
plt.setp(f.axes, yticks=[])
plt.tight_layout(h_pad=2)

# RESUMO POR CLUSTER

In [ ]:
resumo_cluster

# Cluster 0

In [ ]:
# Cluster 0

labels = 'Masculino', 'Feminino'
sizes = [472, 483]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Solteiro', 'Não Solterio'
sizes = [451, 504]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Outro', 'Ensino Médio', 'Graduado', 'Universitário'
sizes = [125, 714, 12, 104]
explode = (0, 0.08, 0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Desempregado', 'Funcionário', 'Gestão ou Autônomo'
sizes = [190, 739, 26]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Média', 'Pequena', 'Grande'
sizes = [266, 481, 208]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

# Cluster 1

In [ ]:
# Cluster 1

labels = 'Feminino', 'Masculino'
sizes = [139, 318]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Não Solteiro', 'Solterio'
sizes = [199, 258]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Outro', 'Ensino Médio', 'Graduado', 'Universitário'
sizes = [53, 279, 16, 109]
explode = (0, 0.08, 0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Desempregado', 'Funcionário', 'Gestão ou Autônomo'
sizes = [31, 281, 145]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Média', 'Pequena', 'Grande'
sizes = [69, 212, 176]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

# Cluster 2

In [ ]:
# Cluster 2

labels = 'Masculino', 'Feminino'
sizes = [222, 263]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Solteiro', 'Não Solterio'
sizes = [236, 249]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Outro', 'Ensino Médio', 'Graduado', 'Universitário'
sizes = [93, 354, 4, 34]
explode = (0, 0.08, 0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Funcionário', 'Desempregado', 'Gestão ou Autônomo'
sizes = [70, 412, 3]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Média', 'Pequena', 'Grande'
sizes = [26, 432, 27]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

# Cluster 3

In [ ]:
resumo_cluster

In [ ]:
# Cluster 3

labels = 'Feminino', 'Masculino'
sizes = [29, 74]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Não Solteiro', 'Solterio'
sizes = [41, 62]
explode = (0, 0.08)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Outro', 'Universitário', 'Graduado', 'Ensino Médio'
sizes = [16, 44, 4, 39]
explode = (0, 0.08, 0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Funcionário', 'Gestão ou Autônomo', 'Desempregado'
sizes = [23, 80, 0]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()

In [ ]:
labels = 'Pequena', 'Grande', 'Média'
sizes = [7, 56, 40]
explode = (0, 0.08, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 

plt.show()